In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder, PolynomialFeatures, RobustScaler
from sklearn.decomposition import PCA
from sklearn.feature_selection import VarianceThreshold
from sklearn.impute import SimpleImputer
from keras.models import Sequential
from keras.layers import Dense, Dropout, LeakyReLU
from keras.optimizers import Adamax
from keras.callbacks import EarlyStopping

# Load data
df_train = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')

# Separate features and target
X_train = df_train.drop('price_doc', axis=1)
y_train = df_train['price_doc']

# Separate features and target for test set
X_test = df_test.copy()


In [9]:

# Step 1: One-Hot Encoding for categorical columns
object_cols_train = X_train.select_dtypes(include=['object']).columns
X_train = pd.get_dummies(X_train, columns=object_cols_train, drop_first=True)

# Step 2: PCA Transformation to reduce dimensionality to 30 components
pca_train = PCA(n_components=10)
X_train = pca_train.fit_transform(X_train)

# Step 3: Variance Thresholding to remove features with low variance
variance_filter_train = VarianceThreshold(threshold=0.01)
X_train = pd.DataFrame(variance_filter_train.fit_transform(X_train), columns=X_train.columns[variance_filter_train.get_support()])

# Step 4: Robust Scaling
scaler_train = RobustScaler()
X_train = scaler_train.fit_transform(X_train)

# poly_train = PolynomialFeatures(degree=3)
# X_train = poly_train.fit_transform(X_train)

MemoryError: Unable to allocate 355. MiB for an array with shape (256, 181507) and data type float64

In [ ]:

# Data preprocessing for test set
X_test = X_test.drop(['row ID'], axis=1)

object_cols_test = X_test.select_dtypes(include=['object']).columns
X_test = pd.get_dummies(X_test, columns=object_cols_test, drop_first=True)
X_test = pca_train.transform(X_test)

X_test = pd.DataFrame(X_test, columns=X_test.columns)

variance_filter_test = VarianceThreshold(threshold=0.01)
X_test = pd.DataFrame(variance_filter_test.fit_transform(X_test), columns=X_test.columns[variance_filter_test.get_support()])

X_test = scaler_train.transform(X_test)

X_test = pca_train.transform(X_test)

# X_test = poly_train.transform(X_test)


In [ ]:
# Model
n_features = X_train.shape[1]
model = Sequential()
model.add(Dense(100, input_dim=n_features))
model.add(LeakyReLU(alpha=0.01))
model.add(Dropout(0.2))
model.add(Dense(50))
model.add(LeakyReLU(alpha=0.01))
model.add(Dropout(0.2))
model.add(Dense(25))
model.add(LeakyReLU(alpha=0.01))
model.add(Dropout(0.2))
model.add(Dense(1, activation='linear'))
model.compile(optimizer=Adamax(learning_rate=0.01), loss='mean_squared_error')


In [ ]:

# Model training
early_stopping = EarlyStopping(monitor='loss', patience=3, restore_best_weights=True)
model.fit(X_train, y_train, epochs=100, batch_size=256, callbacks=[early_stopping])


Epoch 1/100

710/710 [==============================] - 4s 4ms/step - loss: 252518474973184.0000
Epoch 2/100
710/710 [==============================] - 3s 4ms/step - loss: 220312041947136.0000
Epoch 3/100
710/710 [==============================] - 3s 4ms/step - loss: 217323902337024.0000
Epoch 4/100
710/710 [==============================] - 3s 4ms/step - loss: 213481114566656.0000
Epoch 5/100
710/710 [==============================] - 3s 4ms/step - loss: 213328626450432.0000
Epoch 6/100
710/710 [==============================] - 2s 4ms/step - loss: 211068987113472.0000
Epoch 7/100
710/710 [==============================] - 3s 4ms/step - loss: 209939091947520.0000
Epoch 8/100
710/710 [==============================] - 3s 4ms/step - loss: 208726870982656.0000
Epoch 9/100
710/710 [==============================] - 3s 4ms/step - loss: 208783745744896.0000
Epoch 10/100
710/710 [==============================] - 3s 4ms/step - loss: 206438978813952.0000
Epoch 11/100
710/710 [================

In [ ]:
# Prediction for test set
y_pred = model.predict(X_test)

# Save predictions to a CSV file
result_df = pd.DataFrame({'row ID': range(1, len(y_pred) + 1), 'price_doc': y_pred.flatten()})
result_df.to_csv('neural_29nov_3_improved.csv', index=False)

2431/2431 [==============================] - 2s 901us/step
